<a href="https://colab.research.google.com/github/hannanabdul55/neural-nets-research-playground/blob/master/Ad_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import json
import re

import torch

# Try import ing external module, if not, pip install the pacage and import 
try:
    from transformers import BertModel, BertTokenizer
except:
    !pip install transformers
    from transformers import BertModel, BertTokenizer

import sklearn
# from sklearn_pandas import DataFrameMapper
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [3]:
# set the device to the used
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Dataset generation
This section can be skipped if you have the preprocessed data in your drive.  

First, we read the CSV file into a Pandas DataFrame.

In [4]:
df = pd.read_csv("en-US.csv")

## Some useful information of the Dataframe

In [5]:
print(f"Total number of ads: {df.shape[0]}")
print(f"Number of political ads: {(df['political'] >= 1).sum()}")
print(f"Percentage of political ads: {(df['political'] >= 1).sum()/df.shape[0]}")


targets = []

def update_targets(target):
    '''
        This method parsees the target JSON file and appends any new target to the targets list.
    '''
    if pd.isna(target):
        return
    target = json.loads(target)
    for t in target:
        if t['target'] not in targets:
            targets.append(t['target'])


# apply the update_targets method to every row of the DataFrame.
df.apply(lambda row: update_targets(row['targets']), axis=1)
print(f"possible values of targets: \n{targets}")

Total number of ads: 222186
Number of political ads: 130106
Percentage of political ads: 0.5855724483090744
possible values of targets: 
['List', 'Audience Owner', 'Mobile App', 'Like', 'MinAge', 'Age', 'Country', 'Location Granularity', 'Location Type', 'Retargeting', 'Region', 'Activity on the Facebook Family', 'Interest', 'Language', 'Segment', 'Education', 'MaxAge', 'Gender', 'State', 'City', 'Website', 'Custom Audience Match Key', 'Job Title', 'Relationship Status', 'Agency', 'Employer', 'Engaged with Content']


In [10]:
df['political'].unique()

array([  0,   2,   1,   4,   3,  12,   5,   6,   8,   7,  18,  10,  11,
        15,  17,   9,  16,  19,  13,  14,  21,  27,  25,  26,  29,  28,
        22,  20,  35,  31,  38,  23,  32,  33,  40,  36,  34,  30,  43,
        45,  24,  37,  51,  57,  48,  52,  55,  41,  46,  49, 158,  97,
       128, 137,  83, 127, 101,  69,  42,  39,  72,  47,  71,  54,  50,
        91,  89,  86,  82,  44,  53, 100,  87,  64,  60,  66,  59,  56,
       107,  93,  79,  81,  78,  75,  73,  58,  61,  88,  65,  74,  62,
        68,  95,  84,  98,  96,  63, 119,  85, 115, 105, 351,  94, 103,
       117, 389, 114,  77, 265,  70,  90, 298, 113, 125, 366, 360, 401,
       300,  80, 335, 364, 319, 371, 361,  67, 353, 257, 237, 269, 272,
       138, 253, 243, 215, 192, 255, 490, 402, 411, 417, 277, 383, 339,
       394, 314, 333, 405, 474, 439, 443, 410, 457, 139, 303, 376, 475,
       452, 440, 477, 481, 418, 446])

## Preprocess
We want to parse the `target` column and use that information to construct additional feature columns ot every data based on the target information. 

In [11]:
def update_row_with_targets(row):
    if pd.isna(row['targets']):
        return row
    tgt = json.loads(row['targets'])
    for t in tgt:
        if 'segment' in t:
            row[t['target']] = t['segment']
    return row

 
df = df.append(pd.DataFrame(columns=targets)).apply(lambda row: update_row_with_targets(row), axis=1)

# Preprocess dataset

In [ ]:
df.drop(columns=['Unnamed: 0','id','html', 'thumbnail', 'targeting', 'suppressed',
       'targets','thumbnail', 'created_at', 'updated_at', 'lang', 'images','listbuilding_fundraising_proba',
       'page_id', 'List','entities', 'page', 'lower_page','targetings', 'Mobile App'], inplace=True)

In [12]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

df['message'] = df.apply(lambda row: cleanhtml(row['message']), axis=1)

In [15]:
df.to_csv('ad_processed.csv')

# Save processed CSV file to drive
Please note that to do this, you will have to connect drive to this notebook. Then, mention the entire path (including file name) from the root of your drive. An example is given as the default. Use the form in the following cell to give the path.

In [ ]:
#@title File path if you want to save csv to Drive
path_in_drive = "UMass/datasets/processed_ad_dataset.csv" #@param {type:"string"}
full_path = f"/content/drive/My Drive/{path_in_drive}"
df.to_csv(full_path)

Great! You now have the entire preprocessed Ad dataset saved to your drive!

# [WIP]Modeling

In [ ]:
# Run this ONLY if you're starting the notebook from here. Please set teh drive path in the previous cell
df = pd.read_csv(full_path)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (27,28,31,32,33,42,45,46,47,48,49,50,51) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
sample_ids = tokenizer.encode('Hello this is Abdul')
print(f"Hello this is Abdul == {sample_ids}")

Hello this is Abdul == [101, 7592, 2023, 2003, 10298, 102]


In [ ]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
model.eval()

In [ ]:
def get_embedding(sent):
    toks = tokenizer.encode(sent)
    with torch.no_grad():
        out = model(input_ids=torch.LongTensor(toks).unsqueeze(0))
    hidden_states = out[2]
    sent_embed = torch.mean(hidden_states[-1], dim=1).squeeze()
    return sent_embed
